To obtain the data with the rent prices, we will do web-scrapping from Idealista, a real estate platform that operates primarily in Spain. By scraping data from Idealista, we can access up-to-date information on rental prices across different regions.

We encountered some access problems to do web-scrapping directly from the url, so we extracted the html for the tables that contained the information that we wanted as of 29/04

In [1]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
def extract_prices_from_file(html_file_path, city_name):
    with open(html_file_path, "r", encoding="utf-8") as file:
        html = file.read()

    soup = BeautifulSoup(html, "html.parser")
    rows = soup.select("tr.table__row")

    data = []
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 2:
            date = cells[0].get_text(strip=True)
            raw_price = cells[1].get_text(strip=True)
            match = re.search(r"[\d,.]+", raw_price)
            price_text = match.group().replace(",", ".")
            data.append({
                    "date": date,
                    "price": price_text,
                    "city": city_name
                })

    return pd.DataFrame(data)


In [3]:
path = os.getcwd()

# Define the directory containing the HTML files
directory = os.path.join(path, "data/html_files")

In [4]:
print(path)

/Users/raquel/Documents/master/social_data/social_data_assignments/final_project


In [5]:
files = [table for table in os.listdir(directory) if table.endswith(".html")]
names = [table.split("_")[2].split(".")[0] for table in files]

In [6]:
# Create a DataFrame for each file
dataframes = []
for i, file in enumerate(files):
    df = extract_prices_from_file(directory+"/"+file, names[i])
    print(f"Extracted {len(df)} rows from {file}")
    dataframes.append(df)

# Concatenate all DataFrames into one
df_all = pd.concat(dataframes, ignore_index=True)



Extracted 231 rows from table_prices_palma.html
Extracted 231 rows from table_prices_valencia.html
Extracted 231 rows from table_prices_barcelona.html
Extracted 231 rows from table_prices_madrid.html


In [7]:
df_all = df_all[df_all["price"] != "."]
df_all["price"] = df_all["price"].astype(float)

In [8]:
# save the DataFrame to a CSV file
output_file = os.path.join(path, "data_clean", "prices.csv")
df_all.to_csv(output_file, index=False)